In [ ]:
target = 'area'

---

# Define the metrics

**RMSE** 

RMSE is the most popular evaluation metric used in regression problems. It follows an assumption that error are unbiased and follow a normal distribution.

# Dependencies

In [ ]:
#Import required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression

# Load and describe data

In [ ]:
# path = 'forestfires.csv'
path = "../input/forest-fires-data-set/forestfires.csv"
df = pd.read_csv(path)

df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe().T

# Missing value treatment

In [ ]:
df.isna().sum().sum()

# Exploratory Data Analysis
   We will try out the following analysis on our dataset
   - Univariate 
   - Bivariate 
   - Multivariate

In [ ]:
plt.rcParams["figure.figsize"] = 9,5

## Univariate analysis



### Let's begin with the target variable, `Area`

In [ ]:
plt.figure(figsize=(16,5))
print("Skew: {}".format(df[target].skew()))
print("Kurtosis: {}".format(df[target].kurtosis()))
ax = sns.kdeplot(df[target],shade=True,color='g')
plt.xticks([i for i in range(0,1200,50)])
plt.show()

In [ ]:
ax = sns.boxplot(df[target])

**Few observations:**

- The data is highly skewed with a value of +12.84 and huge kurtosis value of 194.

- It even tells you that majority of the forest fires do not cover a large area, most of the damaged area is under 50 hectares of land.

- We can apply tranformation to fix the skewnesss and kurtosis, however we will have to inverse transform before submitting the output.

- Outlier Check: There are 4 outlier instances in our area columns but the questions is should we drop it or not? (Will get back to this in the outlier treatment step)

In [ ]:
# Outlier points
y_outliers = df[abs(zscore(df[target])) >= 3 ]
y_outliers

### Independent columns 

In [ ]:
dfa = df.drop(columns=target)
cat_columns = dfa.select_dtypes(include='object').columns.tolist()
num_columns = dfa.select_dtypes(exclude='object').columns.tolist()

cat_columns,num_columns

### Categorical columns 

In [ ]:
# analyzing categorical columns
plt.figure(figsize=(16,10))
for i,col in enumerate(cat_columns,1):
    plt.subplot(2,2,i)
    sns.countplot(data=dfa,y=col)
    plt.subplot(2,2,i+2)
    df[col].value_counts(normalize=True).plot.bar()
    plt.ylabel(col)
    plt.xlabel('% distribution per category')
plt.tight_layout()
plt.show()    

1. It is interesting to see that abnormally high number of the forest fires occur in the month of `August`
and `September`.

2. In the case of day, the days `Friday` to `Monday` have higher proportion of cases. (However, no strong indicators)

### Numerical Columns

In [ ]:
plt.figure(figsize=(18,40))
for i,col in enumerate(num_columns,1):
    plt.subplot(8,4,i)
    sns.kdeplot(df[col],color='g',shade=True)
    plt.subplot(8,4,i+10)
    df[col].plot.box()
plt.tight_layout() 
plt.show()
num_data = df[num_columns]
pd.DataFrame(data=[num_data.skew(),num_data.kurtosis()],index=['skewness','kurtosis'])

Outliers, Skewness and kurtosis (high positive or negative) was observed in the following columns:
1. FFMC
2. ISI
3. rain

## Bivariate analysis with our target variable

In [ ]:
print(df['area'].describe(),'\n')
print(y_outliers)

In [ ]:
# a categorical variable based on forest fire area damage
# No damage, low, moderate, high, very high
def area_cat(area):
    if area == 0.0:
        return "No damage"
    elif area <= 1:
        return "low"
    elif area <= 25:
        return "moderate"
    elif area <= 100:
        return "high"
    else:
        return "very high"

df['damage_category'] = df['area'].apply(area_cat)
df.head()

### Categorical columns

In [ ]:
cat_columns

In [ ]:
for col in cat_columns:
    cross = pd.crosstab(index=df['damage_category'],columns=df[col],normalize='index')
    cross.plot.barh(stacked=True,rot=40,cmap='hot')
    plt.xlabel('% distribution per category')
    plt.xticks(np.arange(0,1.1,0.1))
    plt.title("Forestfire damage each {}".format(col))
plt.show()

- Previously we had observed that `August` and `September` had the most number of forest fires. And from the above plot of `month`, we can understand few things
    - Most of the fires in August were low (< 1 hectare).
    - The very high damages(>100 hectares) happened in only 3 months - august,july and september.
 
- Regarding fire damage per day, nothing much can be observed. Except that, there were no ` very high` damaging fires on Friday and on Saturdays it has been reported most.

### Numerical columns

In [ ]:
plt.figure(figsize=(20,40))
for i,col in enumerate(num_columns,1):
    plt.subplot(10,1,i)
    if col in ['X','Y']:
        sns.swarmplot(data=df,x=col,y=target,hue='damage_category')
    else:
        sns.scatterplot(data=df,x=col,y=target,hue='damage_category')
plt.show()

## Multivariate analysis

In [ ]:
selected_features = df.drop(columns=['damage_category','day','month']).columns
selected_features

In [ ]:
sns.pairplot(df,hue='damage_category',vars=selected_features)
plt.show()

# Outlier treatment

We had observed outliers in the following columns:
1. area 
2. FFMC
2. ISI
3. rain

In [ ]:
out_columns = ['area','FFMC','ISI','rain']

However, the above outliers are not error values so we cannot remove it. 

In order to minimize the effect of outliers in our model we will transform the above features. 

**Ref:** https://humansofdata.atlan.com/2018/03/when-delete-outliers-dataset/

# Preparing the data for modelling
Thing which we can cover here
- Encoding the categorical columns 

In [ ]:
df = pd.get_dummies(df,columns=['day','month'],drop_first=True)

- Data transformations like `log,root,inverse,exponential`,etc

In [ ]:
print(df[out_columns].describe())
np.log1p(df[out_columns]).skew(), np.log1p(df[out_columns]).kurtosis()

In [ ]:
# FFMC and rain are still having high skew and kurtosis values, 
# since we will be using Linear regression model we cannot operate with such high values
# so for FFMC we can remove the outliers in them using z-score method
mask = df.loc[:,['FFMC']].apply(zscore).abs() < 3

# Since most of the values in rain are 0.0, we can convert it as a categorical column
df['rain'] = df['rain'].apply(lambda x: int(x > 0.0))

df = df[mask.values]
df.shape

In [ ]:
out_columns.remove('rain')
df[out_columns] = np.log1p(df[out_columns])

In [ ]:
df[out_columns].skew()

In [ ]:
# we will use this dataframe for building our ML model
df_ml = df.drop(columns=['damage_category']).copy()

# Linear Regression

In [ ]:
X = df.drop(columns=['area','damage_category'])
y = df['area']

In [1]:
lr = LinearRegression() # Type code here
lr.fit(X,y)

print(f'Intercept: {lr.intercept_}')
print(f'R^2 score: {lr.score(X, y)}')
pd.DataFrame({"Coefficients": lr.coef_}, index=X.columns)

NameError: name 'LinearRegression' is not defined

# Improving Model

**Dropping columns to improve accuracy:**
    
By checking high Variance inflation factor and p-value we will decide whether to keep the column or drop it.

> R^2 = 1 - SSE(Sum of Square of Residuals)/SST (Sum of square Total)

Just by dropping constant we got a huge bump in adjusted R2 from `2.5%` to `40.6%`.

In [ ]:
X = df.drop(columns=['area','damage_category'])
y = df['area']

In [ ]:
def check_stats(X,y):
    vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print(pd.DataFrame({'vif': vif}, index=X.columns).sort_values(by="vif",ascending=False)[:10])
    lin_reg = sm.OLS(y,X).fit()
    print(lin_reg.summary())
check_stats(X,y)

In [ ]:
X.drop(columns=['FFMC'],inplace=True)
# check_stats(X,y)

In [ ]:
X.drop(columns=['Y'],inplace=True)
# check_stats(X,y)

In [ ]:
X.drop(columns=['month_jul'],inplace=True)
# check_stats(X,y)

In [ ]:
X.drop(columns=['day_thu'],inplace=True)
# check_stats(X,y)

In [ ]:
X.drop(columns=['day_mon'],inplace=True)
# check_stats(X,y)

In [ ]:
X.drop(columns=['month_aug'],inplace=True)
check_stats(X,y)

Similarly, you can continue to optimize the model.

Our Prob (F-statistic) has improved from 0.0558 to 2.20e-48. As the value is less than 0.05, the model becomes more significant.